In [1]:
# ISOT Fake News Detection Model

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
import re
import time
from lime.lime_text import LimeTextExplainer
from sklearn.model_selection import train_test_split
import copy

In [2]:
# function to perform minimal text preprocessing for transformer models
# arguments:
# text - input text to preprocess (string)
# max_length - maximum number of words to keep (default: 512)
#
# returns preprocessed text string
def preprocess_text(text, max_length=512):
    # step 1: handle edge cases
    if not isinstance(text, str):
        return ""
    
    # step 2: clean encoding issues
    # remove non-ascii characters that could cause problems
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # step 3: normalize text format
    # replace multiple whitespace characters with single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # step 4: truncate oversized inputs
    # transformer models have context length limits
    words = text.split()
    if len(words) > max_length:
        text = ' '.join(words[:max_length])
        
    return text

In [3]:
# function to load and preprocess WELFake dataset
# arguments:
# welfake_path - path to WELFake_Dataset.csv
# return - tuple of (train_df, valid_df, test_df) with preprocessed data
def load_welfake_dataset(welfake_path="data/welfake_dataset/WELFake_Dataset.csv"):
    # step 1: load the raw dataset
    print(f"Loading WELFake dataset from {welfake_path}...")
    welfake_df = pd.read_csv(welfake_path)

    # step 2: rename/reshape columns
    welfake_df['statement'] = welfake_df['title'] + " " + welfake_df['text'].fillna("")
    welfake_df['label'] = welfake_df['label']  # Already binary (1=real, 0=fake)
    
    # step 3: keep only needed columns
    welfake_df = welfake_df[['statement', 'label']]
    
    # step 4: apply preprocessing
    print("Applying text preprocessing...")
    welfake_df['statement'] = welfake_df['statement'].apply(preprocess_text)

    # step 5: do train/valid/test split
    train_df, temp_df = train_test_split(welfake_df, test_size=0.2, random_state=42)
    valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
    
    # step 6: print dataset statistics
    print(f"WELFake dataset statistics:")
    print(f"  Total: {len(welfake_df)} samples")
    print(f"  Train: {len(train_df)} samples")
    print(f"  Valid: {len(valid_df)} samples")
    print(f"  Test: {len(test_df)} samples")
    
    # step 7: check class balance
    print(f"\nClass distribution:")
    print(f"  Overall: {welfake_df['label'].value_counts().to_dict()}")
    print(f"  Train: {train_df['label'].value_counts().to_dict()}")
    print(f"  Valid: {valid_df['label'].value_counts().to_dict()}")
    print(f"  Test: {test_df['label'].value_counts().to_dict()}")

    return train_df, valid_df, test_df

In [4]:
# dataset class for ISOT fake news detection
class FakeNewsDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        # step 1: store dataframe and extract necessary columns
        self.df = df
        self.texts = df['statement'].tolist()
        self.labels = df['label'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        # step 1: get text and label for the index
        text = self.texts[idx]
        label = self.labels[idx]
        
        # step 2: tokenize the text
        encodings = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # step 3: return the encodings and label
        return {
            'input_ids': encodings['input_ids'].squeeze(),
            'attention_mask': encodings['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [5]:
# compute metrics for model evaluation
# arguments:
# eval_pred - tuple of (predictions, labels)
# return - dictionary with evaluation metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    return {
        'accuracy': accuracy_score(labels, predictions),
        'precision': precision_score(labels, predictions, zero_division=0),
        'recall': recall_score(labels, predictions),
        'f1': f1_score(labels, predictions),
        'roc_auc': roc_auc_score(labels, predictions) if len(np.unique(labels)) > 1 else 0
    }

In [6]:
# function to load and preprocess ISOT dataset
# arguments:
# isot_paths - tuple of paths to (fake_news_path, true_news_path)
# return - tuple of (train_df, valid_df, test_df) with preprocessed data
def load_isot_dataset(isot_paths=("data/isot_dataset/Fake.csv", "data/isot_dataset/True.csv")):
    # Handle case where isot_paths is a directory path
    if isinstance(isot_paths, str):
        fake_path = f"{isot_paths}/Fake.csv"
        true_path = f"{isot_paths}/True.csv"
    else:
        # Handle case where isot_paths is a tuple of paths
        fake_path, true_path = isot_paths
    
    print(f"Loading ISOT dataset from {fake_path} and {true_path}...")
    
    # Load fake news
    fake_df = pd.read_csv(fake_path)
    fake_df['label'] = 0  # 0 for fake
    
    # Load true news
    true_df = pd.read_csv(true_path)
    true_df['label'] = 1  # 1 for real
    
    # step 2: combine datasets
    isot_df = pd.concat([fake_df, true_df], ignore_index=True)
    
    # step 3: create statement column (title + text)
    isot_df['statement'] = isot_df['title'] + " " + isot_df['text'].fillna("")
    
    # step 4: keep only needed columns
    isot_df = isot_df[['statement', 'label']]
    
    # step 5: apply preprocessing
    print("Applying text preprocessing...")
    isot_df['statement'] = isot_df['statement'].apply(preprocess_text)

    # step 6: do train/valid/test split
    train_df, temp_df = train_test_split(isot_df, test_size=0.2, random_state=42)
    valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
    
    # step 7: print dataset statistics
    print(f"ISOT dataset statistics:")
    print(f"  Total: {len(isot_df)} samples")
    print(f"  Train: {len(train_df)} samples")
    print(f"  Valid: {len(valid_df)} samples")
    print(f"  Test: {len(test_df)} samples")
    
    # step 8: check class balance
    print(f"\nClass distribution:")
    print(f"  Overall: {isot_df['label'].value_counts().to_dict()}")
    print(f"  Train: {train_df['label'].value_counts().to_dict()}")
    print(f"  Valid: {valid_df['label'].value_counts().to_dict()}")
    print(f"  Test: {test_df['label'].value_counts().to_dict()}")

    return train_df, valid_df, test_df

In [7]:
from torch.utils.data import WeightedRandomSampler
import copy

def curriculum_learning_pipeline(
    isot_path="data/isot_dataset/",
    welfake_path="data/welfake_dataset/WELFake_Dataset.csv",
    model_name="distilbert-base-uncased",
    replay_fraction=0.5,  # Increased from 0.2 to 0.5
    phase1_lr=5e-5,
    phase2_lr=3e-6,  # Slightly reduced
    batch_size=16,
    phase1_epochs=3,
    phase2_epochs=2,
    seed=42,
    output_dir="./models/curriculum_balanced",
    eval_steps=500,
    save_steps=500
):
    """
    Curriculum learning pipeline with balanced mixed batch replay strategy to prevent catastrophic forgetting.
    
    Args:
        isot_path: Path to ISOT dataset
        welfake_path: Path to WELFake dataset
        model_name: Pretrained model to use
        replay_fraction: Fraction of ISOT samples to keep in replay buffer
        phase1_lr: Learning rate for phase 1 (ISOT)
        phase2_lr: Learning rate for phase 2 (mixed dataset)
        batch_size: Batch size for training
        phase1_epochs: Number of epochs for phase 1
        phase2_epochs: Number of epochs for phase 2
        seed: Random seed
        output_dir: Directory to save models
        eval_steps: Steps between evaluations
        save_steps: Steps between checkpoints
    
    Returns:
        tuple: (trainer, model, tokenizer, results_dict)
    """
    # Set random seeds for reproducibility
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    # Step 1: Load datasets
    print("Loading datasets...")
    isot_train_df, isot_valid_df, isot_test_df = load_isot_dataset(isot_path)
    welfake_train_df, welfake_valid_df, welfake_test_df = load_welfake_dataset(welfake_path)
    
    # Step 2: Load tokenizer and model
    print(f"Loading {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    
    # Step 3: Phase 1 - Train on ISOT dataset
    print("\n===== PHASE 1: Training on ISOT dataset =====")
    phase1_output_dir = f"{output_dir}/phase1"
    
    # Create PyTorch datasets
    isot_train_dataset = FakeNewsDataset(isot_train_df, tokenizer)
    isot_valid_dataset = FakeNewsDataset(isot_valid_df, tokenizer)
    
    # Training arguments
    training_args_phase1 = TrainingArguments(
        output_dir=phase1_output_dir,
        num_train_epochs=phase1_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=phase1_lr,
        weight_decay=0.01,
        eval_strategy="steps",
        eval_steps=eval_steps,
        save_steps=save_steps,
        logging_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        push_to_hub=False,
        report_to="none",
        seed=seed
    )

    # Define compute metrics function
    def compute_metrics(pred):
        logits, labels = pred.predictions, pred.label_ids
        preds = np.argmax(logits, axis=-1)
        accuracy = accuracy_score(labels, preds)
        f1 = f1_score(labels, preds)
        precision = precision_score(labels, preds)
        recall = recall_score(labels, preds)
        return {
            'accuracy': accuracy,
            'f1': f1,
            'precision': precision,
            'recall': recall
        }
    
    # Create trainer
    trainer_phase1 = Trainer(
        model=model,
        args=training_args_phase1,
        train_dataset=isot_train_dataset,
        eval_dataset=isot_valid_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    # Train the model
    print("Training on ISOT dataset...")
    trainer_phase1.train()
    
    # Save the model from Phase 1
    model_p1 = copy.deepcopy(model)
    
    # Evaluate on ISOT test dataset
    print("Evaluating on ISOT test dataset...")
    isot_test_dataset = FakeNewsDataset(isot_test_df, tokenizer)
    isot_test_results = trainer_phase1.evaluate(isot_test_dataset)
    print(f"ISOT Test Results: {isot_test_results}")
    
    # Step 4: Create ISOT replay buffer for mixed batch training
    print(f"\nCreating ISOT replay buffer with {replay_fraction*100}% of training samples...")
    isot_replay_df = isot_train_df.sample(frac=replay_fraction, random_state=seed)
    print(f"ISOT replay buffer size: {len(isot_replay_df)} samples")
    
    # Step 5: Combine ISOT replay buffer with WELFake training data
    print("Creating mixed dataset for phase 2...")
    mixed_train_df = pd.concat([welfake_train_df, isot_replay_df])
    # Add source column to track data origins
    if 'source' not in welfake_train_df.columns:
        welfake_train_df['source'] = 'welfake'
    if 'source' not in isot_replay_df.columns:
        isot_replay_df['source'] = 'isot'
        
    # We'll need these source values for weighted sampling
    mixed_train_df = pd.concat([welfake_train_df, isot_replay_df])
    mixed_train_df = mixed_train_df.sample(frac=1.0, random_state=seed)  # Shuffle
    
    print(f"Mixed dataset size: {len(mixed_train_df)} samples")
    print(f"  - WELFake samples: {len(welfake_train_df)}")
    print(f"  - ISOT replay samples: {len(isot_replay_df)}")
    
    # Step 6: Create weighted sampler to balance dataset representation
    print("Creating weighted sampler...")
    # Calculate class weights based on inverse frequency
    dataset_counts = mixed_train_df['source'].value_counts()
    total_samples = len(mixed_train_df)
    num_datasets = len(dataset_counts)
    
    # Create sample weights array
    sample_weights = torch.ones(len(mixed_train_df))
    
    # Assign weights to balance dataset representation
    for i, row in enumerate(mixed_train_df.itertuples()):
        source = row.source
        weight = total_samples / (num_datasets * dataset_counts[source])
        sample_weights[i] = weight
    
    # Step 7: Create custom DataLoader with weighted sampling
    mixed_train_dataset = FakeNewsDataset(mixed_train_df, tokenizer)
    welfake_valid_dataset = FakeNewsDataset(welfake_valid_df, tokenizer)
    
    # Create sampler
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    # Create custom DataLoader with weighted sampling
    train_dataloader = DataLoader(
        mixed_train_dataset,
        batch_size=batch_size,
        sampler=sampler
    )
    
    # Step 8: Create custom trainer for phase 2 that uses our weighted DataLoader
    class CustomTrainer(Trainer):
        def get_train_dataloader(self):
            return train_dataloader
    
    # Step 9: Phase 2 - Train on mixed dataset with weighted sampling
    print("\n===== PHASE 2: Training on mixed dataset with weighted sampling =====")
    phase2_output_dir = f"{output_dir}/phase2"
    
    # Training arguments for phase 2
    training_args_phase2 = TrainingArguments(
        output_dir=phase2_output_dir,
        num_train_epochs=phase2_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=phase2_lr,  # Lower learning rate for fine-tuning
        weight_decay=0.01,
        eval_strategy="steps",
        eval_steps=eval_steps,
        save_steps=save_steps,
        logging_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        push_to_hub=False,
        report_to="none",
        seed=seed
    )
    
    # Create trainer for phase 2 with weighted sampling
    trainer_phase2 = CustomTrainer(
        model=model,  # Continue with the same model from phase 1
        args=training_args_phase2,
        # train_dataset still needed for some trainer functions
        train_dataset=mixed_train_dataset,
        eval_dataset=welfake_valid_dataset,  # Primary validation on WELFake
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    # Train the model on mixed dataset with weighted sampling
    print("Training on mixed dataset with weighted sampling...")
    trainer_phase2.train()
    
    # Step 10: Evaluate the final model on both datasets
    print("\n===== FINAL EVALUATION =====")
    
    # Evaluate on ISOT test dataset
    print("Evaluating on ISOT test dataset...")
    isot_final_results = trainer_phase2.evaluate(isot_test_dataset)
    print(f"ISOT Final Results: {isot_final_results}")
    
    # Evaluate on WELFake test dataset
    print("Evaluating on WELFake test dataset...")
    welfake_test_dataset = FakeNewsDataset(welfake_test_df, tokenizer)
    welfake_final_results = trainer_phase2.evaluate(welfake_test_dataset)
    print(f"WELFake Final Results: {welfake_final_results}")
    
    # Step 11: Compare with Phase 1 model on both datasets
    print("\n===== COMPARING PHASE 1 MODEL WITH FINAL MODEL =====")
    
    # Restore Phase 1 model
    model_p1_eval = model_p1.eval()
    
    # Create trainer for Phase 1 model evaluation
    trainer_p1_eval = Trainer(
        model=model_p1_eval,
        compute_metrics=compute_metrics,
        args=TrainingArguments(output_dir="./tmp_eval", report_to="none")
    )
    
    # Evaluate Phase 1 model on WELFake
    print("Evaluating Phase 1 model on WELFake test dataset...")
    welfake_p1_results = trainer_p1_eval.evaluate(welfake_test_dataset)
    print(f"WELFake Results from Phase 1 model: {welfake_p1_results}")
    
    # Collect all results
    results = {
        "isot_phase1": isot_test_results,
        "isot_final": isot_final_results,
        "welfake_final": welfake_final_results,
        "welfake_phase1": welfake_p1_results
    }
    
    print("\nCurriculum learning with weighted sampling complete!")
    
    return trainer_phase2, model, tokenizer, results

In [8]:
trainer, model, tokenizer, results = curriculum_learning_pipeline()

Loading datasets...
Loading ISOT dataset from data/isot_dataset//Fake.csv and data/isot_dataset//True.csv...
Applying text preprocessing...
ISOT dataset statistics:
  Total: 44898 samples
  Train: 35918 samples
  Valid: 4490 samples
  Test: 4490 samples

Class distribution:
  Overall: {0: 23481, 1: 21417}
  Train: {0: 18748, 1: 17170}
  Valid: {0: 2348, 1: 2142}
  Test: {0: 2385, 1: 2105}
Loading WELFake dataset from data/welfake_dataset/WELFake_Dataset.csv...
Applying text preprocessing...
WELFake dataset statistics:
  Total: 72134 samples
  Train: 57707 samples
  Valid: 7213 samples
  Test: 7214 samples

Class distribution:
  Overall: {1: 37106, 0: 35028}
  Train: {1: 29768, 0: 27939}
  Valid: {1: 3667, 0: 3546}
  Test: {1: 3671, 0: 3543}
Loading distilbert-base-uncased...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



===== PHASE 1: Training on ISOT dataset =====
Training on ISOT dataset...


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.000200,0.003604,0.999332,0.999300,0.999533,0.999066
1000,0.014800,0.006252,0.998664,0.998601,0.997207,1.000000
1500,0.000100,0.000146,1.000000,1.000000,1.000000,1.000000
2000,0.000100,0.000065,1.000000,1.000000,1.000000,1.000000
2500,0.009100,0.000162,1.000000,1.000000,1.000000,1.000000
3000,0.000000,0.000024,1.000000,1.000000,1.000000,1.000000


Evaluating on ISOT test dataset...


ISOT Test Results: {'eval_loss': 0.0023900421801954508, 'eval_accuracy': 0.999554565701559, 'eval_f1': 0.9995247148288974, 'eval_precision': 1.0, 'eval_recall': 0.9990498812351544, 'eval_runtime': 43.7319, 'eval_samples_per_second': 102.671, 'eval_steps_per_second': 6.426, 'epoch': 1.3363028953229399}

Creating ISOT replay buffer with 50.0% of training samples...
ISOT replay buffer size: 17959 samples
Creating mixed dataset for phase 2...
Mixed dataset size: 75666 samples
  - WELFake samples: 57707
  - ISOT replay samples: 17959
Creating weighted sampler...

===== PHASE 2: Training on mixed dataset with weighted sampling =====
Training on mixed dataset with weighted sampling...


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.683600,0.857497,0.209206,0.065225,0.081725,0.054268
1000,0.668400,0.824024,0.253293,0.181210,0.204741,0.162531
1500,0.629800,0.721696,0.367531,0.376265,0.377296,0.375239
2000,0.609600,0.722120,0.351865,0.350965,0.357466,0.344696
2500,0.587100,0.698964,0.380702,0.403047,0.395178,0.411235
3000,0.573100,0.770592,0.345765,0.332248,0.345294,0.320153
3500,0.579300,0.672636,0.365035,0.365651,0.371517,0.359967
4000,0.581900,0.670912,0.370026,0.375309,0.378431,0.372239



===== FINAL EVALUATION =====
Evaluating on ISOT test dataset...


ISOT Final Results: {'eval_loss': 0.519503653049469, 'eval_accuracy': 0.9367483296213809, 'eval_f1': 0.9367764915405165, 'eval_precision': 0.8814411395056556, 'eval_recall': 0.9995249406175772, 'eval_runtime': 26.9353, 'eval_samples_per_second': 166.696, 'eval_steps_per_second': 10.432, 'epoch': 0.8456659619450317}
Evaluating on WELFake test dataset...
WELFake Final Results: {'eval_loss': 0.6870463490486145, 'eval_accuracy': 0.371915719434433, 'eval_f1': 0.390748957913137, 'eval_precision': 0.38582049920339884, 'eval_recall': 0.39580495777717245, 'eval_runtime': 48.9, 'eval_samples_per_second': 147.526, 'eval_steps_per_second': 9.223, 'epoch': 0.8456659619450317}

===== COMPARING PHASE 1 MODEL WITH FINAL MODEL =====
Evaluating Phase 1 model on WELFake test dataset...


WELFake Results from Phase 1 model: {'eval_loss': 7.855900764465332, 'eval_model_preparation_time': 0.001, 'eval_accuracy': 0.17604657610202384, 'eval_f1': 0.009993337774816789, 'eval_precision': 0.012858979854264894, 'eval_recall': 0.008172160174339417, 'eval_runtime': 50.0132, 'eval_samples_per_second': 144.242, 'eval_steps_per_second': 18.035}

Curriculum learning with weighted sampling complete!


In [9]:
def predict_text(text, model, tokenizer, threshold=0.5):
    """
    Predict if a given news article is real or fake
    """
    # Preprocess text
    processed_text = preprocess_text(text)
    
    # Tokenize
    inputs = tokenizer(
        processed_text,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    # Move inputs to the same device as the model
    device = model.device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Convert to probabilities
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    fake_prob = probabilities[0, 0].item()
    real_prob = probabilities[0, 1].item()
    
    # Get prediction
    predicted_class = 1 if real_prob > threshold else 0
    label = "REAL" if predicted_class == 1 else "FAKE"
    
    return {
        "prediction": label,
        "fake_probability": fake_prob,
        "real_probability": real_prob,
        "confidence": max(fake_prob, real_prob)
    }